# CrewAI + LangWatch Scenarios Demo

This notebook demonstrates how to use **LangWatch Scenarios** to test **CrewAI** multi-agent systems through AI-powered simulation testing.

## What You'll Learn

1. **Multi-Agent Systems**: How to build collaborative AI agent teams with CrewAI
2. **AI Testing**: Using AI agents to test other AI agents with LangWatch scenarios
3. **Realistic Scenarios**: Creating comprehensive test scenarios for complex interactions
4. **Quality Evaluation**: Using custom judges to evaluate agent performance

## Prerequisites

Make sure you have:
- OpenAI API key set in your environment
- All required packages installed (`pip install -r requirements.txt`)
- Basic understanding of AI agents and testing concepts

## Setup and Imports

In [ ]:
import os
import sys
import asyncio
import json
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Check API key
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Please set your OPENAI_API_KEY in the .env file")
else:
    print("✅ OpenAI API key found")

In [ ]:
# Import our demo components
from agents.customer_service_crew import CustomerServiceCrew
from adapters.crew_adapter import create_crew_adapter
from scenarios.judges.custom_judges import (
    create_quality_judge, 
    create_technical_judge, 
    create_escalation_judge
)

# Import LangWatch scenarios
import scenario

print("✅ All imports successful")

## Part 1: Understanding CrewAI Multi-Agent System

Let's start by exploring the CrewAI customer service system we've built.

In [ ]:
# Initialize the customer service crew
crew = CustomerServiceCrew()

# Explore the crew structure
crew_info = crew.get_crew_info()
print("🤖 Customer Service Crew Structure:")
print(json.dumps(crew_info, indent=2))

In [ ]:
# Test the crew with a simple inquiry
print("📞 Testing Customer Service Crew")
print("Customer: I can't log into my account")
print("\n🤖 Crew Response:")

response = crew.handle_inquiry(
    "I can't log into my account", 
    customer_id="DEMO_001"
)

print(response)

## Part 2: Setting Up LangWatch Scenarios

Now let's configure LangWatch scenarios to test our CrewAI system.

In [ ]:
# Configure LangWatch scenarios
scenario.configure(
    testing_agent=scenario.TestingAgent(
        model=os.getenv("SIMULATOR_MODEL", "openai/gpt-4o-mini")
    )
)

# Create the CrewAI adapter for LangWatch
crew_adapter = create_crew_adapter()

print("✅ LangWatch scenarios configured")
print("✅ CrewAI adapter created")

## Part 3: Running Basic Scenarios

Let's run some basic scenarios to test our customer service system.

In [ ]:
# Basic customer service scenario
async def run_basic_scenario():
    print("🧪 Running Basic Customer Service Scenario")
    
    result = await scenario.run(
        name="basic login troubleshooting",
        description="""
        User is having trouble logging into their account. They're not particularly 
        tech-savvy but are cooperative and willing to follow instructions. They have 
        their login credentials ready and access to their email.
        """,
        agents=[
            crew_adapter,
            scenario.UserSimulatorAgent(),
            scenario.JudgeAgent(criteria=[
                "Agent should ask relevant troubleshooting questions",
                "Agent should provide clear, step-by-step instructions",
                "Agent should be patient and helpful",
                "Agent should offer multiple solutions if the first doesn't work"
            ])
        ],
        max_turns=8
    )
    
    return result

# Run the scenario
basic_result = await run_basic_scenario()

print(f"\n📊 Scenario Result: {'✅ PASSED' if basic_result.success else '❌ FAILED'}")
print(f"💬 Messages exchanged: {len(basic_result.messages)}")
print(f"📝 Feedback: {basic_result.feedback}")

In [ ]:
# Let's examine the conversation that took place
print("🔍 Conversation Analysis:")
print("=" * 50)

for i, message in enumerate(basic_result.messages):
    role = message.get('role', 'unknown')
    content = message.get('content', '')
    
    if role == 'user':
        print(f"\n👤 Customer: {content}")
    elif role == 'assistant':
        print(f"\n🤖 Agent: {content}")
    
    if i >= 10:  # Limit output for readability
        remaining = len(basic_result.messages) - i - 1
        if remaining > 0:
            print(f"\n... ({remaining} more messages)")
        break

## Part 4: Advanced Scenarios with Custom Judges

Now let's use custom judges to evaluate specific aspects of the conversation.

In [ ]:
# Scenario with custom quality judge
async def run_quality_evaluation_scenario():
    print("🎯 Running Quality Evaluation Scenario")
    
    result = await scenario.run(
        name="customer service quality evaluation",
        description="""
        Customer is frustrated about a billing issue that has been ongoing for weeks.
        They're not angry but are clearly stressed and need empathetic, professional help.
        """,
        agents=[
            crew_adapter,
            scenario.UserSimulatorAgent(),
            create_quality_judge()  # Our custom quality judge
        ],
        max_turns=10
    )
    
    return result

quality_result = await run_quality_evaluation_scenario()

print(f"\n📊 Quality Evaluation: {'✅ PASSED' if quality_result.success else '❌ FAILED'}")
print(f"💬 Messages: {len(quality_result.messages)}")

In [ ]:
# Parse and display the quality evaluation results
try:
    # The custom judge returns JSON evaluation
    judge_feedback = quality_result.feedback
    
    # Try to parse as JSON if it's a string
    if isinstance(judge_feedback, str):
        try:
            evaluation_data = json.loads(judge_feedback)
        except json.JSONDecodeError:
            evaluation_data = {"raw_feedback": judge_feedback}
    else:
        evaluation_data = judge_feedback
    
    print("🎯 Quality Evaluation Results:")
    print(json.dumps(evaluation_data, indent=2))
    
except Exception as e:
    print(f"Could not parse evaluation results: {e}")
    print(f"Raw feedback: {quality_result.feedback}")

## Part 5: Technical Support Scenario

Let's test how the system handles technical support requests.

In [ ]:
# Technical support scenario
async def run_technical_scenario():
    print("🔧 Running Technical Support Scenario")
    
    result = await scenario.run(
        name="API integration support",
        description="""
        Developer is trying to integrate the company's API into their application. 
        They're experiencing authentication issues and getting error codes they don't 
        understand. They're technically competent but new to this specific API.
        """,
        agents=[
            crew_adapter,
            scenario.UserSimulatorAgent(),
            create_technical_judge()  # Our custom technical judge
        ],
        max_turns=12
    )
    
    return result

technical_result = await run_technical_scenario()

print(f"\n📊 Technical Scenario: {'✅ PASSED' if technical_result.success else '❌ FAILED'}")
print(f"💬 Messages: {len(technical_result.messages)}")
print(f"📝 Technical Evaluation: {technical_result.feedback[:200]}...")

## Part 6: Escalation Scenario

Let's test how the system handles escalation situations.

In [ ]:
# Escalation scenario
async def run_escalation_scenario():
    print("📈 Running Escalation Scenario")
    
    result = await scenario.run(
        name="customer escalation handling",
        description="""
        Customer has been trying to resolve an issue for weeks and is frustrated.
        They want to speak to a manager and are considering canceling their service.
        The issue is complex and requires escalation to higher-level support.
        """,
        agents=[
            crew_adapter,
            scenario.UserSimulatorAgent(),
            create_escalation_judge()  # Our custom escalation judge
        ],
        max_turns=10
    )
    
    return result

escalation_result = await run_escalation_scenario()

print(f"\n📊 Escalation Scenario: {'✅ PASSED' if escalation_result.success else '❌ FAILED'}")
print(f"💬 Messages: {len(escalation_result.messages)}")

In [ ]:
# Display escalation evaluation
try:
    escalation_feedback = escalation_result.feedback
    if isinstance(escalation_feedback, str):
        try:
            escalation_data = json.loads(escalation_feedback)
            print("📈 Escalation Evaluation:")
            print(json.dumps(escalation_data, indent=2))
        except json.JSONDecodeError:
            print(f"Escalation feedback: {escalation_feedback}")
    else:
        print(f"Escalation evaluation: {escalation_feedback}")
except Exception as e:
    print(f"Error displaying escalation results: {e}")

## Part 7: Scripted Scenario Example

Let's create a scripted scenario to test specific conversation flows.

In [ ]:
# Scripted scenario for error recovery
async def run_scripted_scenario():
    print("📝 Running Scripted Error Recovery Scenario")
    
    result = await scenario.run(
        name="scripted error recovery",
        description="""
        Test how the agent recovers from providing incorrect information.
        The script forces the agent to make a mistake, then tests recovery.
        """,
        agents=[
            crew_adapter,
            scenario.UserSimulatorAgent(),
            scenario.JudgeAgent(criteria=[
                "Agent should acknowledge the mistake when corrected",
                "Agent should apologize for the incorrect information",
                "Agent should provide correct information promptly",
                "Agent should not make excuses"
            ])
        ],
        script=[
            scenario.user("I need help with my billing"),
            scenario.agent("I can help with that. I see you have a Premium plan for $99/month."),
            scenario.user("That's not right, I have the Basic plan for $29/month"),
            scenario.proceed()  # Let the scenario continue naturally
        ],
        max_turns=8
    )
    
    return result

scripted_result = await run_scripted_scenario()

print(f"\n📊 Scripted Scenario: {'✅ PASSED' if scripted_result.success else '❌ FAILED'}")
print(f"💬 Messages: {len(scripted_result.messages)}")
print(f"📝 Feedback: {scripted_result.feedback}")

## Part 8: Results Summary and Analysis

Let's summarize all our test results and analyze the performance.

In [ ]:
# Compile all results
all_results = {
    "basic_scenario": {
        "success": basic_result.success,
        "messages": len(basic_result.messages),
        "feedback": basic_result.feedback
    },
    "quality_evaluation": {
        "success": quality_result.success,
        "messages": len(quality_result.messages),
        "feedback": quality_result.feedback
    },
    "technical_support": {
        "success": technical_result.success,
        "messages": len(technical_result.messages),
        "feedback": technical_result.feedback
    },
    "escalation_handling": {
        "success": escalation_result.success,
        "messages": len(escalation_result.messages),
        "feedback": escalation_result.feedback
    },
    "scripted_scenario": {
        "success": scripted_result.success,
        "messages": len(scripted_result.messages),
        "feedback": scripted_result.feedback
    }
}

# Calculate summary statistics
total_scenarios = len(all_results)
passed_scenarios = sum(1 for result in all_results.values() if result["success"])
total_messages = sum(result["messages"] for result in all_results.values())
avg_messages = total_messages / total_scenarios

print("📊 Test Results Summary")
print("=" * 50)
print(f"Total Scenarios: {total_scenarios}")
print(f"Passed: {passed_scenarios} ✅")
print(f"Failed: {total_scenarios - passed_scenarios} ❌")
print(f"Success Rate: {(passed_scenarios/total_scenarios)*100:.1f}%")
print(f"Total Messages: {total_messages}")
print(f"Average Messages per Scenario: {avg_messages:.1f}")

print("\n📋 Individual Results:")
for scenario_name, result in all_results.items():
    status = "✅" if result["success"] else "❌"
    print(f"  {scenario_name}: {status} ({result['messages']} messages)")

In [ ]:
# Save results to file
import datetime

results_file = project_root / "results" / f"notebook_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
results_file.parent.mkdir(exist_ok=True)

with open(results_file, 'w') as f:
    json.dump(all_results, f, indent=2, default=str)

print(f"💾 Results saved to: {results_file}")

## Part 9: Key Insights and Takeaways

Based on our testing, here are the key insights:

In [ ]:
print("🎯 Key Insights from Testing")
print("=" * 50)

insights = [
    "🤖 Multi-Agent Collaboration: CrewAI enables sophisticated agent collaboration",
    "🧪 AI-Powered Testing: LangWatch scenarios provide realistic testing environments",
    "📊 Custom Evaluation: Specialized judges can evaluate domain-specific criteria",
    "📝 Scripted Control: Scripted scenarios allow testing specific conversation flows",
    "🔄 Iterative Improvement: Results provide actionable feedback for agent improvement",
    "⚡ Scalable Testing: Automated scenarios can test many edge cases efficiently",
    "🎭 Realistic Simulation: UserSimulatorAgent creates believable customer interactions",
    "📈 Quality Metrics: Quantitative evaluation enables systematic improvement"
]

for insight in insights:
    print(f"  {insight}")

print("\n🚀 Next Steps:")
next_steps = [
    "Expand scenario coverage to include more edge cases",
    "Implement continuous testing in CI/CD pipeline",
    "Create domain-specific judges for your use case",
    "Add performance and load testing scenarios",
    "Integrate with monitoring and alerting systems",
    "Build regression testing suite for agent updates"
]

for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

## Conclusion

This notebook demonstrated how to:

1. **Build Multi-Agent Systems** with CrewAI for complex customer service scenarios
2. **Test AI Agents with AI** using LangWatch scenarios for realistic evaluation
3. **Create Custom Judges** for domain-specific evaluation criteria
4. **Use Scripted Scenarios** for controlled testing of specific flows
5. **Analyze Results** to identify areas for improvement

The combination of CrewAI and LangWatch scenarios provides a powerful framework for building and testing production-ready AI agent systems. The AI-powered testing approach scales much better than manual testing and can uncover edge cases that might be missed in traditional testing approaches.

### Resources for Further Learning

- [CrewAI Documentation](https://docs.crewai.com/)
- [LangWatch Scenarios Documentation](https://scenario.langwatch.ai/)
- [AI Agent Testing Best Practices](../docs/best-practices.md)
- [Advanced Integration Patterns](../docs/advanced-patterns.md)

Happy building! 🚀